In [14]:
#imports
import requests as re
import json
import os
import sqlite3
import pandas as pd

In [17]:
tracking = pd.read_csv("/Users/madelinetrumbauer/Local Desktop/F '21/SI 485/Tracking.csv")
bulk = pd.read_csv("/Users/madelinetrumbauer/Local Desktop/F '21/SI 485/bulk.csv")

In [18]:
smithsonian_df= pd.concat([bulk, tracking['Scientific Name']])
#print(smithsonian_df)
#print(smithsonian_df)

#change index col, species, to the name species
smithsonian_df.rename(columns={0:"species"}, inplace = True)

#drop first row of text descriptions of each column
smithsonian_df = smithsonian_df.tail(-1)

In [21]:
def make_a_query(key,query):
    base_url = 'https://api.si.edu/openaccess/api/v1.0/search'
    params_dict={'api_key':key, 'q':query}
    first_search = re.get(base_url, params_dict)
    first_result = json.loads(first_search.text)
    return first_result
def title(result):
    dict_of_titles = {}
    for i in range(len(result['response']['rows'])):
        title= result['response']['rows'][i]['content']['descriptiveNonRepeating']['title']['content']
        title_string=json.dumps(title)
        dict_of_titles[i] = title_string
    return dict_of_titles
def scientific_name(result):
    dict_of_sci_name = {}
    for i in range(len(result['response']['rows'])):
        sci_name = result['response']['rows'][i]['content']['indexedStructured']['scientific_name'][0]
        sci_name_string=json.dumps(sci_name)
        dict_of_sci_name[i] = sci_name_string
    return dict_of_sci_name
def locality(first_result):
    locality_dict = {}
    for i in range(len(first_result['response']['rows'])):
        if "name" in first_result['response']['rows'][i]['content']['freetext'] and "place" in first_result['response']['rows'][i]['content']['freetext'] and 'geoLocation' in first_result['response']['rows'][i]['content']['indexedStructured']:
            name = first_result['response']['rows'][i]['content']['freetext']['name'][0]['content']
            place = first_result['response']['rows'][i]['content']['freetext']['place'][0]['content']
            if 'Other' in first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]:
                    other = first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]['Other']['content']
                    locality_dict[i] = name, place, other 
            else:
                locality_dict[i] = name, place
        elif "name" in first_result['response']['rows'][i]['content']['freetext'] and "place" in first_result['response']['rows'][i]['content']['freetext'] and 'geoLocation' not in first_result['response']['rows'][i]['content']['indexedStructured']:
                name = first_result['response']['rows'][i]['content']['freetext']['name'][0]['content']
                place = first_result['response']['rows'][i]['content']['freetext']['place'][0]['content']
                locality_dict[i] = name, place
        elif "name" in first_result['response']['rows'][i]['content']['freetext'] and "place" not in first_result['response']['rows'][i]['content']['freetext'] and 'geoLocation' in first_result['response']['rows'][i]['content']['indexedStructured']:
                name = first_result['response']['rows'][i]['content']['freetext']['name'][0]['content']
                if 'Other' in first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]:
                    other = first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]['Other']['content']
                    locality_dict[i] = name, other 
                else:
                     locality_dict[i] = name
        elif "name" not in first_result['response']['rows'][i]['content']['freetext'] and "place" in first_result['response']['rows'][i]['content']['freetext'] and 'geoLocation' in first_result['response']['rows'][i]['content']['indexedStructured']:
                place = first_result['response']['rows'][i]['content']['freetext']['place'][0]['content']
                if 'Other' in first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]:
                    other = first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]['Other']['content']
                    locality_dict[i] = place, other 
                else:
                     locality_dict[i] = place
        elif "name" in first_result['response']['rows'][i]['content']['freetext']:
            name = first_result['response']['rows'][i]['content']['freetext']['name'][0]['content']
            locality_dict[i] = name
        elif "place" in first_result['response']['rows'][i]['content']['freetext']:
            place = first_result['response']['rows'][i]['content']['freetext']['place'][0]['content']
            locality_dict[i] = place
        elif 'geoLocation' in first_result['response']['rows'][i]['content']['indexedStructured']:
            if 'Other' in first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]:
                other = first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]['Other']['content']
                locality_dict[i] = other 
        else:
            locality_dict[i] = float("NaN")
        # locality_list.append(locality)
    return locality_dict
def location(first_result):
    location_dict = {}
    for i in range(len(first_result['response']['rows'])):
        if'geoLocation' in first_result['response']['rows'][i]['content']['indexedStructured']:
            if 'L4' in first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]:
                county = first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]['L4']['content']
                county_string = json.dumps(county)
            else:
                 county_string = float("NaN")
            if 'L3' in first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]:
                state = first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]['L3']['content']
                state_string = json.dumps(state)
            else:
                 state_string = float("NaN")
            if 'L2' in first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]:
                country = first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]['L2']['content']
                country_string = json.dumps(country)
            else:
                 country_string = float("NaN")
            if 'L1' in first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]:
                continent = first_result['response']['rows'][i]['content']['indexedStructured']['geoLocation'][0]['L1']['content']
                continent_string = json.dumps(continent)
            else:
                 continent_string = float("NaN")
            # location_string = county_string + ", " + state_string + ", " + country_string + ", " + continent_string
            location_dict[i] = county_string, state_string, country_string, continent_string
        else:
            location_dict[i] = float("NaN")
    return location_dict
#get year
def year(first_result):
    year_dict = {}
    for i in range(len(first_result['response']['rows'])):
        if 'date' in first_result['response']['rows'][i]['content']['indexedStructured']:
            year = first_result['response']['rows'][i]['content']['indexedStructured']['date']
            year_dict[i] = year
        else: year_dict[i] = float("NaN")
    return year_dict
def main():
    #smithsonian to build df
    intermediary_df = pd.DataFrame(columns = ['species', 'title', 'locality', 'location', 'year'])
    for i in smithsonian_df['species']:
        cool = make_a_query('11icycZDd63MfM3OrfwyQwrpJatRz5mFy37cnBo6', i)
        species_title = title(cool)
        species_locality = locality(cool)
        species_location = location(cool)
        species_year = year(cool)
    # # of rows already rows in the intermediary_df + j in square
        for j in species_title:
            intermediary_df = intermediary_df.append({'species': i, 'title':species_title[j], 'locality':species_locality[j], 'location': species_location[j], 'year':species_year[j]}, ignore_index=True)
        #intermediary_df[intermediary_df['locality'].str.match(United States)]
    intermediary_df

if __name__ == "__main__":
    main()  

In [24]:
 intermediary_df = pd.DataFrame(columns = ['species', 'title', 'locality', 'location', 'year'])
for i in smithsonian_df['species']:
    cool = make_a_query('11icycZDd63MfM3OrfwyQwrpJatRz5mFy37cnBo6', i)
    species_title = title(cool)
    species_locality = locality(cool)
    species_location = location(cool)
    species_year = year(cool)
    # # of rows already rows in the intermediary_df + j in square
    for j in species_title:
        intermediary_df = intermediary_df.append({'species': i, 'title':species_title[j], 'locality':species_locality[j], 'location': species_location[j], 'year':species_year[j]}, ignore_index=True)
        #intermediary_df[intermediary_df['locality'].str.match(United States)]
    print(intermediary_df)
    #WORKS

               species                          title  \
0  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
1  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
2  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
3  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
4  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
5  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
6  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
7  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
8  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
9  Cabomba caroliniana  "Cabomba caroliniana A. Gray"   

                                            locality  \
0                                                NaN   
1                             Quiet streams, marshes   
2  (78 - Southeastern U.S.A., Caroliniana. In ori...   
3  (78 - Southeastern U.S.A., Mouth of Dogue Cree...   
4  (78 - Southeastern U.S.A., Dyke, Fairfax, Virg...   
5  (78 - Southeastern U.S.A., Southe

                  species                                              title  \
0     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
1     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
2     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
3     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
4     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
5     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
6     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
7     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
8     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
9     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
10  Myriophyllum spicatum                         "Myriophyllum spicatum L."   
11  Myriophyllum spicatum               

                  species                                              title  \
0     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
1     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
2     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
3     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
4     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
5     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
6     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
7     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
8     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
9     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
10  Myriophyllum spicatum                         "Myriophyllum spicatum L."   
11  Myriophyllum spicatum               

                  species                                              title  \
0     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
1     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
2     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
3     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
4     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
5     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
6     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
7     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
8     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
9     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
10  Myriophyllum spicatum                         "Myriophyllum spicatum L."   
11  Myriophyllum spicatum               

                  species                                              title  \
0     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
1     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
2     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
3     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
4     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
5     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
6     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
7     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
8     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
9     Cabomba caroliniana                      "Cabomba caroliniana A. Gray"   
10  Myriophyllum spicatum                         "Myriophyllum spicatum L."   
11  Myriophyllum spicatum               

...

                  species                                       title  \
0     Cabomba caroliniana               "Cabomba caroliniana A. Gray"   
1     Cabomba caroliniana               "Cabomba caroliniana A. Gray"   
2     Cabomba caroliniana               "Cabomba caroliniana A. Gray"   
3     Cabomba caroliniana               "Cabomba caroliniana A. Gray"   
4     Cabomba caroliniana               "Cabomba caroliniana A. Gray"   
...                   ...                                         ...   
1299       Silurus glanis                            "Silurus glanis"   
1300       Silurus glanis                            "Silurus glanis"   
1301       Silurus glanis                            "Silurus glanis"   
1302       Silurus glanis                                   "Silurus"   
1303       Silurus glanis  "Silurus (Parasilurus) aristotelis Garman"   

                                               locality  \
0                                                   NaN   
1    

                  species                          title  \
0     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
1     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
2     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
3     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
4     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
...                   ...                            ...   
1329    Anguilla rostrata            "Anguilla rostrata"   
1330    Anguilla rostrata            "Anguilla rostrata"   
1331    Anguilla rostrata            "Anguilla rostrata"   
1332    Anguilla rostrata            "Anguilla rostrata"   
1333    Anguilla rostrata            "Anguilla rostrata"   

                                               locality  \
0                                                   NaN   
1                                Quiet streams, marshes   
2     (78 - Southeastern U.S.A., Caroliniana. In ori...   
3     (78 - Southeastern U.S.A., Mouth of D

                         species                          title  \
0            Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
1            Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
2            Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
3            Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
4            Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
...                          ...                            ...   
1339  Misgurnus anguillicaudatus   "Misgurnus anguillicaudatus"   
1340  Misgurnus anguillicaudatus   "Misgurnus anguillicaudatus"   
1341  Misgurnus anguillicaudatus   "Misgurnus anguillicaudatus"   
1342  Misgurnus anguillicaudatus   "Misgurnus anguillicaudatus"   
1343  Misgurnus anguillicaudatus   "Misgurnus anguillicaudatus"   

                                               locality  \
0                                                   NaN   
1                                Quiet streams, marshes   
2     (78 - Southe

                  species                          title  \
0     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
1     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
2     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
3     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
4     Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
...                   ...                            ...   
1367          Salmo salar                  "Salmo salar"   
1368          Salmo salar                  "Salmo salar"   
1369          Salmo salar                  "Salmo salar"   
1370          Salmo salar                  "Salmo salar"   
1371          Salmo salar                  "Salmo salar"   

                                               locality  \
0                                                   NaN   
1                                Quiet streams, marshes   
2     (78 - Southeastern U.S.A., Caroliniana. In ori...   
3     (78 - Southeastern U.S.A., Mouth of D

                    species                          title  \
0       Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
1       Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
2       Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
3       Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
4       Cabomba caroliniana  "Cabomba caroliniana A. Gray"   
...                     ...                            ...   
1397  Cyprinella analostana        "Cyprinella analostana"   
1398  Cyprinella analostana        "Cyprinella analostana"   
1399  Cyprinella analostana        "Cyprinella analostana"   
1400  Cyprinella analostana        "Cyprinella analostana"   
1401  Cyprinella analostana        "Cyprinella analostana"   

                                               locality  \
0                                                   NaN   
1                                Quiet streams, marshes   
2     (78 - Southeastern U.S.A., Caroliniana. In ori...   
3     (78 - Southea